In [31]:
import os 

In [32]:
os.chdir(r"D:\SAMITH\Github\wine-quality-predictor")

In [33]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    """Data Transformation Configuration"""
    root_dir: Path
    data_path: Path

In [34]:
from wine_quality_predictor.constants import *
from wine_quality_predictor.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [36]:
import pandas as pd
import os
import numpy as np
from wine_quality_predictor import logger
from sklearn.model_selection import train_test_split

In [37]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # ===== Outlier Removal and Normalization =====
        df = data.copy()
        features = ['residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'alcohol']

        for feature in features:
            Q1 = df[feature].quantile(0.25)
            Q3 = df[feature].quantile(0.75)
            IQR = Q3 - Q1

            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR

            df = df[(df[feature] >= lower) & (df[feature] <= upper)]

            # Min-max normalization
            minimum = df[feature].min()
            maximum = df[feature].max()
            df[feature] = (df[feature] - minimum) / (maximum - minimum)

        
        # Filling the missing values 
        for col, value in df.items():
            if col != 'type':
                df[col] = df[col].fillna(df[col].mean())

       
        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(df,test_size = 0.25, random_state = 42)



        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted, cleaned, and oversampled data into training and test sets.")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print("Train:", train.shape)
        print("Test:", test.shape)

    

In [38]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-04-30 21:44:41,064: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-30 21:44:41,080: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-30 21:44:41,086: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-30 21:44:41,089: INFO: common: created directory at: artifacts]
[2025-04-30 21:44:41,093: INFO: common: created directory at: artifacts/data_transformation]
[2025-04-30 21:44:41,304: INFO: 1009129410: Splitted, cleaned, and oversampled data into training and test sets.]
[2025-04-30 21:44:41,305: INFO: 1009129410: Train shape: (4515, 13)]
[2025-04-30 21:44:41,309: INFO: 1009129410: Test shape: (1506, 13)]
Train: (4515, 13)
Test: (1506, 13)
